In [ ]:
%pip install pandas

from re import X
import pandas as pd

url='https://drive.google.com/file/d/1oJGoCdcu0J-qyYLQuPHnI6xocdGrq6XK/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
data = pd.read_csv(url)


X = data.iloc[:, 0].tolist()
y = data.iloc[:, -1].to_numpy()

In [ ]:
%pip install tensorflow

import tensorflow as tf

tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X)

X_secuencias = tokenizer.texts_to_sequences(X)
X_features = tf.keras.preprocessing.sequence.pad_sequences(X_secuencias, padding='post')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 10000
max_len = 400

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)

sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index
print(f"Índice de palabras: {word_index}")

data = pad_sequences(sequences, maxlen=max_len,padding='post')

print(f"Secuencias tokenizadas y rellenas: {data}")

In [ ]:
%pip install sklearn
%pip install scikit-learn

from sklearn.model_selection import train_test_split

# Assuming X and y are your features and labels respectively
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

model = Sequential()

embedding_dim = 300
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))

model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(0.3))

model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import os

checkpoint_path = "training_1/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=0)

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32,
    callbacks=[cp_callback, early_stopping]
)


In [ ]:
%pip install matplotlib

import matplotlib.pyplot as plt

# Pérdida
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Curva de pérdida')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

# Precisión
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Curva de precisión')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# Generar probabilidades de las predicciones
y_pred_prob = model.predict(X_test)

# Convertir las probabilidades a clases binarias (0 o 1)
y_pred = (y_pred_prob > 0.5).astype("int32")

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Imprimir la matriz
print("Matriz de confusión:")
print(conf_matrix)


In [ ]:
%pip install seaborn

import seaborn as sns
import matplotlib.pyplot as plt

# Graficar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Negativa", "Positiva"], yticklabels=["Negativa", "Positiva"])
plt.xlabel("Predicción")
plt.ylabel("Etiqueta Real")
plt.title("Matriz de Confusión")
plt.show()

In [ ]:
# Preprocess the input text
input_text = ["What can I say about such a boring and stupid show. The first episode is bad. I watched the second to see if it got any better but it actually got worse. I cannot watch any more."]
input_sequences = tokenizer.texts_to_sequences(input_text)
input_padded = pad_sequences(input_sequences, maxlen=max_len, padding='post')

# Predict using the model
prediction = model.predict(input_padded)
print(prediction)

In [ ]:
model.build(input_shape=(None, max_len))
model.load_weights("training_1/cp.weights.h5")



---

